In [66]:
import requests
from config.config import Client_Secret, Client_ID
import base64
from requests import post , get
import json
from IPython.display import display
import pandas as pd 


In [67]:
#Spotify API isteklerini yetkilendirmek için kullanılan kimlik bilgileri
def get_token():
    auth_string = f"{Client_ID}:{Client_Secret}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes).decode("utf-8"))

    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + auth_base64
    }
    #token alma
    data = {'grant_type': 'client_credentials'}
    response = requests.post(url, headers=headers, data=data)
    json_result = response.json()

    if response.status_code == 200:
        token = json_result['access_token']
        print('Access Token: alındı'+ " "+token)
        return token
    else:
        print('Error:', response.status_code)
        print('Response:', json_result)
        return None

In [68]:
#authorization
def get_auth(token):
     return{ 'Authorization': 'Bearer ' + token}
#Erişim token'ını almak için get_token fonksiyonunu kullanılır.
token=get_token()

Access Token: alındı BQCohb25mQT6Tokehw0fFmN0i1OCaWFNtK59OOO8L94Ob2-1f4SHjHnv5li5CWFU6yo272WzyFnotMizPEmofXmtL-iRyYfKf_fC5dJ5SkfXmnXOeS0


In [69]:
def get_user_playlists(token, user_id):
    # Kullanıcıya ait çalma listesi getiren fonksiyon
    url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    headers=get_auth(token)

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        json_response = response.json()
        playlist_dt = response.json()
        return playlist_dt.get('items', [])
    else:
        print('Error:', response.status_code)
        print('Response:', response.json())
        return None

user_id='31rmq7qts5n52ox2si25ead5iqaq'
playlist_dt=get_user_playlists(token,user_id)

# Tüm çalma listelerini depolamak için 
all_playlists = {'Playlist_owner_id':[], 'Playlist_name':[], 'Playlist_id':[], 'Number_of_songs':[],'Playlist_uri':[]}

# #Çalma listesi varsa
# if response.status_code == 200:
#     for playlist in json_result['items']:
#         print('Playlist ID:', playlist['id'])
#         print('Playlist Name:', playlist['name'])
#         print('Playlist URI:', playlist['uri'])
#         print('Playlist Owner ID:', playlist['owner']['id'])
#         print('-----------------------------')
# else:
#     print('Error:', response.status_code)
#     print('Response:', json_result)

if playlist_dt:
    # Her çalma listesi için gerekli bilgileri çıkart
    for playlist in playlist_dt:
        all_playlists['Playlist_id'].append(playlist['id'])
        all_playlists['Playlist_name'].append(playlist['name'])
        all_playlists['Number_of_songs'].append(playlist['tracks']['total'])
        all_playlists['Playlist_uri'].append(playlist['uri'])
        all_playlists['Playlist_owner_id'].append(playlist['owner']['id'])

# Elde edilen bilgileri DataFrame'e dönüştürma
all_playlists_df = pd.DataFrame(all_playlists)

all_playlists_df


,Playlist_owner_id,Playlist_name,Playlist_id,Number_of_songs,Playlist_uri
0,31rmq7qts5n52ox2si25ead5iqaq,2. Karışık,2gEpmDLiL0RiG86dtYd7Cc,44,spotify:playlist:2gEpmDLiL0RiG86dtYd7Cc
1,31rmq7qts5n52ox2si25ead5iqaq,1. Çalma Listem,7lGk615a7j3e1NqStHea9r,29,spotify:playlist:7lGk615a7j3e1NqStHea9r


In [74]:
def get_playlist_artists(token,playlist_id):
    # Çalma listesindeki parçaları alma
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = get_auth(token)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        tracks = response.json()
        return tracks.get('items', [])

    else:
        print('Error:', response.status_code)
        print('Response:', response.json())
        return None
    

#Çalma listesi id alma
for playlist_id in all_playlists_df['Playlist_id']:
 print(playlist_id)



2gEpmDLiL0RiG86dtYd7Cc
7lGk615a7j3e1NqStHea9r


In [84]:
#Çalma listesindeki her veriyi depolamak için
playlists_item={'Name':[],'id':[] ,'Artist':[], 'Duration':[], 'Release_date':[], 'Popularity':[]}

#Playlistesindeki verileri alma
for playlist_id in all_playlists_df['Playlist_id']:
    play_tracks=get_playlist_artists(token,playlist_id)


    if play_tracks:
            for track in play_tracks:
             playlists_item['Name'].append(track['track']['name'])
             playlists_item['Artist'].append(track['track']['artists'][0]['name'])
             playlists_item['Duration'].append(track['track']['duration_ms'])
             playlists_item['Release_date'].append(track['added_at'])
             playlists_item['id'].append(track['track']['id'])
             playlists_item['Popularity'].append(track['track']['popularity'])

#DataFrame çevğrme
playlists_item_df =pd.DataFrame(playlists_item)
playlists_item_df

            

,Name,id,Artist,Duration,Release_date,Popularity
0,Seninle Ben,3W6rKNbovssiDHqIcXBjlk,Gökhan Türkmen,279005,2023-12-27T12:23:02Z,55
1,Seninle Ben,4mlnEk7KKXiUweyqwwTEjf,Ayna,333760,2023-12-27T12:23:05Z,19
2,Aşkın Kederi,4tA6mbvO5vzFg9S87s3fcC,Manuş Baba,229960,2023-12-27T12:23:13Z,47
3,Susma,5Q4OWZdBISVfWjbtRpIeiB,Aşkın Nur Yengi,324826,2023-12-27T12:23:14Z,42
4,Pamuk,5e84hFQxrRDXsZwScxNIak,Kenan Doğulu,239133,2023-12-27T12:23:16Z,40
...,...,...,...,...,...,...
68,Sadece,5PBCWGudD3lw0Qhk7jLMUo,Kalben,175996,2023-12-27T12:24:28Z,67
69,Kısa Mesafe,7AeN1Miu6Io3ewNUdBzhyy,Rafat Hasanlı,147497,2023-12-27T12:24:38Z,69
70,Giderdi Hoşuma,7LXZEyI1ke06UnmuwG4P8d,Yaşlı Amca,220530,2023-12-27T12:24:41Z,66
71,Galiba - Orijinal Rap Versiyon,6uuQZFYzeUf1NR5tucdjj5,Sagopa Kajmer,300506,2023-12-27T12:24:41Z,65


In [93]:
#DataFrame'i csv dosyasına kaydetme
playlists_item_df.to_csv('playlists_item.csv', index=False)


## Veri Görselleştirme

In [95]:
spotify=pd.read_csv('spotify.csv')

In [97]:
spotify.head()

,Name,id,Artist,Duration,Release_date,Popularity
0,Seninle Ben,3W6rKNbovssiDHqIcXBjlk,Gökhan Türkmen,279005,2023-12-27T12:23:02Z,55
1,Seninle Ben,4mlnEk7KKXiUweyqwwTEjf,Ayna,333760,2023-12-27T12:23:05Z,19
2,Aşkın Kederi,4tA6mbvO5vzFg9S87s3fcC,Manuş Baba,229960,2023-12-27T12:23:13Z,47
3,Susma,5Q4OWZdBISVfWjbtRpIeiB,Aşkın Nur Yengi,324826,2023-12-27T12:23:14Z,42
4,Pamuk,5e84hFQxrRDXsZwScxNIak,Kenan Doğulu,239133,2023-12-27T12:23:16Z,40


##### Çalma listesindeki en popüler şarkılar

In [98]:
high_po = spotify.loc[ spotify ['Popularity' ] >= 60 ].sort_values( by =[ 'Popularity' ] ,ascending = False) 

high_po.head()

,Name,id,Artist,Duration,Release_date,Popularity
44,Fark Ettim,4rbXJipz3CaSQwtG3xXzzT,Semicenk,155406,2023-12-24T10:46:56Z,78
69,Kısa Mesafe,7AeN1Miu6Io3ewNUdBzhyy,Rafat Hasanlı,147497,2023-12-27T12:24:38Z,69
68,Sadece,5PBCWGudD3lw0Qhk7jLMUo,Kalben,175996,2023-12-27T12:24:28Z,67
66,Değmesin Ellerimiz,0RoQ5sk1Z9PH2xki3gv5hO,Model,267702,2023-12-27T12:24:27Z,67
70,Giderdi Hoşuma,7LXZEyI1ke06UnmuwG4P8d,Yaşlı Amca,220530,2023-12-27T12:24:41Z,66


In [ ]:
fig=px.scatter(spotify,x)